In [2]:
import ee

import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date
from datetime import timedelta
import zipfile
import os
import requests

import numpy as np

import geopy.distance
!pip install geopandas
import geopandas  as gpd
from shapely.geometry import Point, LineString, Polygon


# Import specific function 'from_epsg' from fiona module
from fiona.crs import from_epsg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 14.5 MB/s 
     |████████████████████████████████| 16.7 MB 58.3 MB/s 
     |████████████████████████████████| 6.3 MB 53.9 MB/s 


In [4]:
# connect to google drive
from google.colab import drive  

# drive roots
DRIVE_MOUNT = '/content/drive'
drive.mount(DRIVE_MOUNT, force_remount=True)

# set up project directories
PROJECT_DIR = DRIVE_MOUNT + "/My Drive/Volcano_datasets/"
TMP_DIR = PROJECT_DIR + "TempDIR/"
targets = pd.read_excel(PROJECT_DIR + "Sample Eruptions.xlsx")
unique_targets = targets.drop_duplicates(subset=["Volcano Number", "Volcano Name", "Latitude", "Longitude"])

OUTPUT_SHAPE_DIR = PROJECT_DIR+"output_damage_shapes/"

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
def standarize_volcano_name(name):
  return (re.sub(r"[,.]","",re.sub(r"\s+", "_",name)))

In [8]:
# power plants

cutoff_dist_km = 200 

plants = pd.read_csv(PROJECT_DIR + "/power_plant.csv")

output_df = pd.DataFrame(columns=["volc_name","volc_num","plant_name","plant_country","plant_capacity","plant_lat","plant_lon","primary fuel","pv_dist_km"])
for iplant, plant in plants.iterrows():
  for ivolc, volc in unique_targets.iterrows():
    # are they close?
    if (np.abs(volc["Latitude"] - plant["latitude"])< 1 and np.abs(volc["Longitude"] - plant["longitude"])< 1):
      # ok, so how close realy?
      dist = geopy.distance.distance((volc["Latitude"],volc["Longitude"]), (plant["latitude"],plant["longitude"])).kilometers
      if (dist < cutoff_dist_km):
        # close enough!
        output_df = output_df.append({"volc_name": standarize_volcano_name(volc[ "Volcano Name"]),
                          "volc_num": volc[ "Volcano Number"],
                          "plant_name": plant["name"],
                          "plant_country": plant["country_long"],
                          "plant_capacity": plant["capacity_mw"],
                          "plant_lat": plant["latitude"],
                          "plant_lon": plant["longitude"],
                          "primary fuel": plant["primary_fuel"],
                          "pv_dist_km": dist},
                         ignore_index=True)
  print(f"finished plant {iplant} {plant['name']} {plant['country_long']}")

output_df = output_df.sort_values(by = ["volc_name","pv_dist_km"])
output_df.to_csv(PROJECT_DIR + "/volc_power_plant.csv",index=False)
print(f"wrote file: {PROJECT_DIR + '/volc_power_plant.csv'}")



Streaming output truncated to the last 5000 lines.
finished plant 29937 Manning PV 1 United States of America
finished plant 29938 Mansfield Mill United States of America
finished plant 29939 Manteca Land PV United States of America
finished plant 29940 Manti Lower United States of America
finished plant 29941 Manti Upper United States of America
finished plant 29942 Manway Solar Farm United States of America
finished plant 29943 Manzana Wind LLC United States of America
finished plant 29944 Manzano Solar United States of America
finished plant 29945 Maple Lake United States of America
finished plant 29946 Maple Ridge Wind Farm United States of America
finished plant 29947 Maple Solar United States of America
finished plant 29948 Mapleton Community Solar United States of America
finished plant 29949 Maquoketa 1 United States of America
finished plant 29950 Maquoketa 2 United States of America
finished plant 29951 Marathon Electric United States of America
finished plant 29952 Marathon 

In [13]:
# dams

cutoff_dist_km = 200 

dams = gpd.read_file(PROJECT_DIR+"/Dam Data/GOODD_data_unzipped/Data/GOOD2_dams.shp")

output_df = pd.DataFrame(columns=["volc_name","volc_num","dam_id","dam_lat","dam_lon","dv_dist_km"])
for idam, dam in dams.iterrows():
  for ivolc, volc in unique_targets.iterrows():
    # are they close?
    if (np.abs(volc["Latitude"] - dam["Latitud"])< 1 and np.abs(volc["Longitude"] - dam["Longitud"])< 1):
      # ok, so how close realy?
      dist = geopy.distance.distance((volc["Latitude"],volc["Longitude"]), (dam["Latitud"],dam["Longitud"])).kilometers
      if (dist < cutoff_dist_km):
        # close enough!
        output_df = output_df.append({"volc_name": standarize_volcano_name(volc[ "Volcano Name"]),
                          "volc_num": volc[ "Volcano Number"],
                          "dam_id": dam["DAM_ID"],
                          "dam_lat": dam["Latitud"],
                          "dam_lon": dam["Longitud"],
                          "dv_dist_km": dist},
                         ignore_index=True)
  print(f"finished dam {idam} {dam['DAM_ID']}")

output_df = output_df.sort_values(by = ["volc_name","dv_dist_km"])
output_df.to_csv(PROJECT_DIR + "/volc_dam.csv",index=False)
print(f"wrote file: {PROJECT_DIR + '/volc_dam.csv'}")


Streaming output truncated to the last 5000 lines.
finished dam 33668 1033668.0
finished dam 33669 1033669.0
finished dam 33670 1033670.0
finished dam 33671 1033671.0
finished dam 33672 1033672.0
finished dam 33673 1033673.0
finished dam 33674 1033674.0
finished dam 33675 1033675.0
finished dam 33676 1033676.0
finished dam 33677 1033677.0
finished dam 33678 1033678.0
finished dam 33679 1033679.0
finished dam 33680 1033680.0
finished dam 33681 1033681.0
finished dam 33682 1033682.0
finished dam 33683 1033683.0
finished dam 33684 1033684.0
finished dam 33685 1033685.0
finished dam 33686 1033686.0
finished dam 33687 1033687.0
finished dam 33688 1033688.0
finished dam 33689 1033689.0
finished dam 33690 1033690.0
finished dam 33691 1033691.0
finished dam 33692 1033692.0
finished dam 33693 1033693.0
finished dam 33694 1033694.0
finished dam 33695 1033695.0
finished dam 33696 1033696.0
finished dam 33697 1033697.0
finished dam 33698 1033698.0
finished dam 33699 1033699.0
finished dam 33700 10

In [ ]:
SCALE_FACTOR = 1 # conversion factor from the log of the population, to the number of km in radius of the circle

newdata = gpd.GeoDataFrame()
id = 0
for ivol in range(targets.shape[0]):
  volc = targets.iloc[ivol]
  vloc = Point( volc["Longitude"],volc["Latitude"])
  v_name = standardize_volcano_name(volc["Volcano Name"])
  cities = pop_centers[pop_centers["pop_vol_id"] == volc["Volcano Number"]].sort_values(by = 'pop_vol_dist_km')
  for irow, pop in cities.iterrows():
    if pop["Population"] > 0:
      lin_scale = 2**np.floor(np.log2(pop["Population"]))/100000
      log_scale = np.floor(np.log2(pop["Population"]))/10
     
      # draw a pentagon around city
      lin_pts = []
      log_pts = []
      for bearing in range(0,350,72):
        pt =  geopy.distance.distance(kilometers=lin_scale)\
                    .destination((pop["pop_lat"],pop["pop_lon"]), bearing=bearing)[:2]
        lin_pts.append([pt[1],pt[0]])

        log_pt =  geopy.distance.distance(kilometers=log_scale)\
                    .destination((pop["pop_lat"],pop["pop_lon"]), bearing=bearing)[:2]
        log_pts.append([log_pt[1],log_pt[0]])
      lin_poly = Polygon(lin_pts)
      log_poly = Polygon(log_pts)
      newdata.loc[id,'geometry'] = lin_poly
      newdata.loc[id,"volc_name"] = v_name
      newdata.loc[id,"scale"] = 'linear'
      newdata.loc[id,"volc_num"] = volc["Volcano Number"]
      newdata.loc[id,"city_name"] = pop["Name"]
      newdata.loc[id,"c_country"] = pop["pop_Country"]
      newdata.loc[id,"city_pop"] = pop["Population"]
      newdata.loc[id,"cv_dist_km"] = int(pop["pop_vol_dist_km"])
      id = id +1

      newdata.loc[id,'geometry'] = log_poly
      newdata.loc[id,"scale"] = 'log'
      newdata.loc[id,"volc_name"] = v_name
      newdata.loc[id,"volc_num"] = volc["Volcano Number"]
      newdata.loc[id,"erupt_num"] = volc["Eruption Number"]
      newdata.loc[id,"city_name"] = pop["Name"]
      newdata.loc[id,"c_country"] = pop["pop_Country"]
      newdata.loc[id,"city_pop"] = pop["Population"]
      newdata.loc[id,"cv_dist_km"] = int(pop["pop_vol_dist_km"])
      id = id +1
    
print(newdata)
newdata.set_crs('EPSG:3857')
output_file = f'{OUTPUT_SHAPE_DIR}/city_data.shp'
newdata.to_file(output_file)

                                               geometry  \
0     POLYGON ((55.76790 -21.36049, 55.76828 -21.360...   
1     POLYGON ((55.76790 -21.35002, 55.77890 -21.357...   
2     POLYGON ((55.79616 -21.12821, 55.79653 -21.128...   
3     POLYGON ((55.79616 -21.11774, 55.80715 -21.125...   
4     POLYGON ((55.61691 -21.37474, 55.61992 -21.376...   
...                                                 ...   
3337  POLYGON ((-85.36452 12.11985, -85.35142 12.110...   
3338  POLYGON ((-88.60000 13.43352, -88.59982 13.433...   
3339  POLYGON ((-88.60000 13.44327, -88.59034 13.436...   
3340  POLYGON ((-85.28737 12.19083, -85.28701 12.190...   
3341  POLYGON ((-85.28737 12.20131, -85.27688 12.193...   

                  volc_name   scale  volc_num  erupt_num       city_name  \
0     Fournaise_Piton_de_la  linear  233020.0    22343.0  Saint-Philippe   
1     Fournaise_Piton_de_la     log  233020.0    22343.0  Saint-Philippe   
2     Fournaise_Piton_de_la  linear  233020.0    22343.0     Sa